# ``EmissionModel`` functionality

Here we give an overview of the basic functionality of an ``EmissionModel``. Elsewhere we cover [what models are available](premade_models.ipynb), [how to modify a model](modify_models.ipynb), [making your own custom models](custom_models.ipynb), and [combining_models from different components](combined_models.ipynb).

## Getting a model

To use an ``EmissionModel`` model we simply instatiate it with the required arguments, however, which arguments are required depends on the exact operation. 

For extraction operations you need:

- A grid to extract from (``grid``).
- A key to extract (``extract``). For lines, ``line_ids`` must be passed to ``get_lines``, this key is then the label given to the ``LineCollection``. 
- Optionally, an escape fraction to apply which defaults to 0.0 (``fesc`` or ``covering_fraction`` for AGN).

For combination operations you need:

- The a list of models which will be combined (i.e. added) to give the resultant emission (``combine``).

For generation you need:

- A generator class (e.g. a [dust emission model](dust_emission.ipynb)) from which to generate spectra (``generator``). Note, this is not applicable to lines.

For attenuation operations you need:

- The [dust curve](attenuation/attenuation.rst) to apply (``dust_curve``).
- The model to apply the attenuation to (``apply_dust_to``).
- The optical depth to use with the dust curve (``tau_v``).

Masking can be applied alongside any of these operations. Additionally, any number of masks can be combined on the same operation. Each mask is defined by:

- The attribute of the component to mask on (``mask_attr``).
- The threshold of the mask (``mask_thresh``).
- The operator to use when generating the mask, i.e. ``"<"``, ``">"``, ``"<="``, ``">="``, ``"=="``, or ``"!="`` (``mask_op``).

For this demonstration we'll load the semi-complex ``TotalEmission`` premade stellar emisison model. When using more complex [premade models](premade_models.ipynb), with a deeper "tree", more parameters will be required to populate their "child" models deeper in the tree. This tree terminology will become clear as you read on. Don't worry about the specifics here, they are detailed in the docs linked above.

In [ ]:
from synthesizer.emission_models import TotalEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Blackbody
from synthesizer.grid import Grid
from unyt import Myr, kelvin

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

total = TotalEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.67,
    fesc=0.2,
    fesc_ly_alpha=0.7,
    dust_emission_model=Blackbody(temperature=100 * kelvin),
)

In addition to the arguments specfic to each type of model, any model can be passed arguments to define a mask (the emitter attribute to define a mask with, ``mask_attr``; the operator to use in the mask, ``mask_op``; the threshold for the mask, ``mask_thresh``). 

In [ ]:
masked_total = TotalEmission(
    grid=grid,
    label="young_total",
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.67,
    fesc=0.2,
    fesc_ly_alpha=0.7,
    dust_emission_model=Blackbody(temperature=100 * kelvin),
    mask_attr="ages",
    mask_op="<",
    mask_thresh=10 * Myr,
)

In the above case we've got a ``TotalEmission`` model where every child model in the tree has also been masked. We've done this for demonstration purposes, in most (but certainly not all) cases you'll only want specific models masked. This can either be done by [constructing your own models](custom_models.ipynb) or [modifying existing ones](modify_models.ipynb). 

## Printing a summary of an ``EmissionModel``

If we want to see a summary of all the models contained within an instance of an ``EmissionModel`` we simply print the model.

In [ ]:
print(masked_total)

You can see this gives a block describing each model including it's label (in all caps), it's emitter (in brackets after the label), and the operation the model performs followed by the parameters defining the operation.

## Plotting an ``EmissionModel``

For a more detailed view of how models relate to each other we can also visualise the tree itself.

In [ ]:
total.plot_emission_tree()
masked_total.plot_emission_tree()

In the emission tree plot we can see each individual spectra generated by the model and how they relate. 
- Solid lines denote combinations of spectra into another. 
- Dashed lines denote an attenuation of the child spectra to produce the parent. 
- Dotted lines denote a relationship between the parent generator and it's child spectra used to scale the generated spectra (not required for all generators). 
- Blue boxes have no mask applied. 
- Green boxes have a mask applied to them (which we only have for the masked model we made). 
- Square boxes are an extraction operation. 
- Boxes with rounded corners are generation, dust attenuation or combination operations. 

You'll notice the extraction operations are always leaves in the tree.

## Extracting models

An emission model can be treated like a dictionary. If we want to get a model from somewhere in the tree we simply index with the model label.

In [ ]:
sub_model = total["reprocessed"]
sub_model.plot_emission_tree()